In [ ]:
# imports
import pandas as pd

import sqlalchemy
from sqlalchemy.orm import Session
from sqlalchemy import create_engine 

NOTES

1) Import web scraping - 17S? Vote counter? GM sheets? 

## PART 1: EXTRACTION

source: https://docs.google.com/spreadsheets/d/1CxrgdT4Xd8J0N3CuVmkVRryMeIdm93H8rzv8-YlP0kw/

In [2]:
# Player data ('Data Test' tab)
player_path = "DataTest.csv"
playerdata = pd.read_csv(player_path)
playerdata.head()

In [57]:
# Game data ('Game Stats' tab)
game_path = "GameStats.csv"
gamedata = pd.read_csv(game_path)
gamedata.head()

,Game Stats,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 166,Unnamed: 167,Unnamed: 168,Unnamed: 169,Unnamed: 170,Unnamed: 171,Unnamed: 172,Unnamed: 173,Unnamed: 174,Unnamed: 175
0,NaN,LG1,LG2,LG3,LG4,QF1,LG5,MR1,LG6,QF2,...,QF46,LG68,MR44,LG69,QF47,LG70,MR45,LG71,QF48,MR46
1,# of Players,16,29,22,30,12,20,27,25,15,...,21,20,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Total Rank,132,16,59,13,166,84,29,43,140,...,73,84,162,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,# of Spectators,3,2,5,1,11,2,4,5,9,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Rank,85,97,65,118,37,97,73,65,44,...,139,139,139,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## PART 2: TRANSFORMATION
### 2A: Player data

In [4]:
playerdata.columns

Index(['ID', 'Player', 'Alignment', 'Faction Outcome', 'First Hit', 'Last Hit',
       '# of Hits', 'Death/\n Survival', 'Inactive', 'GM', 'Mod', 'Spec',
       'Game Type', 'Game #', 'Unnamed: 14', 'Broken', 'Role', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24'],
      dtype='object')

In [5]:
# Select useful columns (last few are irrelevant)
pdata = playerdata[["Player", "Alignment", "Faction Outcome", "First Hit", "Last Hit", "# of Hits", 
             "Death/\n Survival", "Inactive", 'GM', 'Mod', 'Spec', 'Game Type', 'Game #', 
             'Unnamed: 14', 'Broken', 'Role']]

# Rename columns for accuracy or simplicity
pdata = pdata.rename(columns = {"Faction Outcome": "fOutcome", "First Hit": "Hit1", "Last Hit": "HitL",
                       "# of Hits": "HitNum", "Death/\n Survival": "Death", "Game Type": "format",
                       "Game #": "GameNum", "Unnamed: 14": "Game"})

,Player,Alignment,fOutcome,Hit1,HitL,HitNum,Death,Inactive,GM,Mod,Spec,format,GameNum,Game,Broken,Role
0,Meta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,LG,1.0,LG1,NaN,GM
1,Cracknut,G,L,1.0,1.0,1.0,E,NaN,NaN,NaN,NaN,LG,1.0,LG1,NaN,Tineye
2,Edgedancer,G,L,2.0,2.0,1.0,L,NaN,NaN,NaN,NaN,LG,1.0,LG1,NaN,Cop
3,Mailliw73,G,L,4.0,4.0,1.0,L,NaN,NaN,NaN,NaN,LG,1.0,LG1,NaN,Regular
4,Aether,E,W,4.0,4.0,1.0,V,NaN,NaN,NaN,NaN,LG,1.0,LG1,NaN,Smoker


In [21]:
# Fill in NaN values
pdata = pdata.fillna({"Alignment": 0, 
                       "fOutcome": 0, 
                      "Hit1": "-", 
                      "HitL": "-", 
                      "HitNum": "-", 
                      "Inactive":"N", 
                      "GM":"N", 
                      "Mod": "N", 
                      "Spec": "N", 
                      "Broken": "N", 
                      "Role": "-"})
print(len(pdata))
pdata.head()

5763


,Player,Alignment,fOutcome,Hit1,HitL,HitNum,Death,Inactive,GM,Mod,Spec,format,GameNum,Game,Broken,Role
0,Meta,0,0,,,,NaN,N,Y,N,N,LG,1.0,LG1,N,GM
1,Cracknut,G,L,1,1,1,E,N,N,N,N,LG,1.0,LG1,N,Tineye
2,Edgedancer,G,L,2,2,1,L,N,N,N,N,LG,1.0,LG1,N,Cop
3,Mailliw73,G,L,4,4,1,L,N,N,N,N,LG,1.0,LG1,N,Regular
4,Aether,E,W,4,4,1,V,N,N,N,N,LG,1.0,LG1,N,Smoker


In [24]:
# separate out mod/spec/GM into separate table

# get indices of all non-players
nonplayers_list = []
it = pdata.iterrows()
i = 0
while i != 5672:
    curr = next(it)
    
    ifmod = curr[1]["Mod"]
    ifGM = curr[1]["GM"]
    ifspec = curr[1]["Spec"]
    
    if (ifmod == "Y") or (ifGM == "Y") or (ifspec == "Y"):
        nonplayers_list.append(curr[0])
    
    i += 1

print(len(nonplayers_list))

# @TODO: SIMPLIFY etc

1445


In [25]:
# pull relevant columns and filter by index list above
nonplayers = pdata.filter(["Player", "GM", "Mod", "Spec", "format", "GameNum", "Game", "Broken"], axis = 1)
nonplayers = nonplayers.filter(nonplayers_list, axis = 0)

nonplayers.head()

# to clean further: combine GM/mod/spec into one column, don't need 'Game' probably
# Maybe separate into separate GM/Mod/Spec tables?

,Player,GM,Mod,Spec,format,GameNum,Game,Broken
0,Meta,Y,N,N,LG,1.0,LG1,N
17,Windy,N,N,Y,LG,1.0,LG1,N
18,Weiry,N,N,Y,LG,1.0,LG1,N
19,Alvron,N,N,Y,LG,1.0,LG1,N
20,Meta,Y,N,N,LG,2.0,LG2,N


In [26]:
# take out nonplayers to create players dataframe
playerlist = [j for j in range(5763) if j not in nonplayers_list]

players = pdata.filter(playerlist, axis = 0)
players = players.drop(['GM', 'Mod', 'Spec'], axis = 1)

players.head()

4318

### 2B: Game data

In [67]:
gamedata.head()

gdata0 = gamedata.fillna('NA')

In [68]:
# need to transpose entire thing and set first column to be the column headers
# this is a MESS
# maybe when getting data initially, read 1 row first and then feed to pandas? solves header problem partly

cols = gdata0["Game Stats"]

cols

0                  NA
1        # of Players
2          Total Rank
3     # of Spectators
4                Rank
           ...       
73            Faction
74               Draw
75       Free For All
76             No One
77               Misc
Name: Game Stats, Length: 78, dtype: object

In [69]:
gdata1 = gdata0.transpose()

gdata2 = gdata1.rename(columns = cols)

gdata2.head()

,NA,# of Players,Total Rank,# of Spectators,Rank,# of E. Kills,# of Neutral Evil Kils,# of V. Kills,# of Lynches,# of FF Deaths,...,Faction/Free For All,NA,NA,Eliminator,Village,Faction,Draw,Free For All,No One,Misc
Game Stats,NA,# of Players,Total Rank,# of Spectators,Rank,# of E. Kills,# of Neutral Evil Kils,# of V. Kills,# of Lynches,# of FF Deaths,...,Faction/Free For All,NA,NA,Eliminator,Village,Faction,Draw,Free For All,No One,Misc
Unnamed: 1,LG1,16,132,3,85,5,0,1,4,0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Unnamed: 2,LG2,29,16,2,97,14,0,2,7,2,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Unnamed: 3,LG3,22,59,5,65,3,0,1,7,3,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Unnamed: 4,LG4,30,13,1,118,4,0,4,8,4,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [ ]:
gdata2.set_index()

In [72]:
gdata2.columns

gdata3 = gdata2[['NA', '# of Players', 'Total Rank', '# of Spectators', 'Rank',
       '# of E. Kills', '# of Neutral Evil Kils', '# of V. Kills',
       '# of Lynches', '# of FF Deaths', '# of Neutral Deaths',
       '# of Inactive Deaths', '# of Deaths', 'Rank', '# of Eliminators',
       'Eliminator Percentage', 'Eliminators Lynched', "Eliminators Vig'd",
       'Thread Reply Count', 'Thread Reply Rank', 'Setting', 'Game Complexity',
       '# of Cycles', 'Winning Team', '# of Survivors', 'Total Rank',
       'Survivors']]

gdata3.head()

,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,Thread Reply Count,Thread Reply Rank,Setting,Game Complexity,# of Cycles,Winning Team,# of Survivors,Total Rank,Total Rank,Survivors
Game Stats,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,Thread Reply Count,Thread Reply Rank,Setting,Game Complexity,# of Cycles,Winning Team,# of Survivors,Total Rank,Total Rank,Survivors
Unnamed: 1,LG1,Gamma,bartbug,little wilson,Oudeis/Darnam,loganmathewjohnson,NA,NA,NA,NA,...,462,79,Cosmere,Basic,4,Eliminator,6,132,86,Claincy
Unnamed: 2,LG2,Quiver,little wilson,Windy,NA,NA,NA,NA,NA,NA,...,"1,232",11,Cosmere,Standard,11,Eliminator,4,16,49,Aether
Unnamed: 3,LG3,Edgedancer,Mailliw73,Gamma,Swimmingly,SirVarrock,Lanscaper,Chid/Newan,NA,NA,...,739,41,Cosmere,Basic,7,Village,8,59,112,Meta
Unnamed: 4,LG4,bartbug,PorridgeBrick,Herowannabe,Macen,Chid/Newan,Adolin Dustbringer,nard1993,Sphinx,NA,...,"1,029",22,Cosmere,Standard,9,Village,9,13,123,Mailliw73


## PART 3: LOAD

In [46]:
players.columns

Index(['Player', 'Alignment', 'fOutcome', 'Hit1', 'HitL', 'HitNum', 'Death',
       'Inactive', 'format', 'GameNum', 'Game', 'Broken', 'Role'],
      dtype='object')

In [50]:
players['Alignment'].value_counts()

G    2426
E     631
0     622
F     281
N     220
B      51
M      32
S      21
C      20
D      14
Name: Alignment, dtype: int64

In [47]:
g_players = players.groupby('Player')

In [48]:
g_players["format"].value_counts()

Player             format
18thShard          LG         2
A Budgie           LG         2
                   MR         2
                   QF         1
A Joe in the Bush  LG        30
                             ..
xxGaea             MR         6
                   LG         3
                   QF         3
yafeshan           LG         3
                   MR         1
Name: format, Length: 789, dtype: int64

In [49]:
players["Alignment"].value_counts()

G    2426
E     631
0     622
F     281
N     220
B      51
M      32
S      21
C      20
D      14
Name: Alignment, dtype: int64

In [53]:
# Create Engine for census data
path = "GameData.sqlite"
engine = create_engine(f"sqlite:///{path}")
conn = engine.connect()

In [54]:
session = Session(bind=engine)

In [55]:
players.to_sql("Players", conn, if_exists = 'replace')

In [56]:
nonplayers.to_sql("Nonplayers", conn, if_exists = 'replace')

In [ ]:
# import matplotlib
# from matplotlib import style
# style.use('fivethirtyeight')
# import matplotlib.pyplot as plt

